In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

policy_raw = pd.read_csv('policy_growth.csv')
exposure_raw = pd.read_csv('exposure_growth_data.csv')

def clean_column(series):
    return (
        series.astype(str)  # Convert to string (in case of mixed types)
        .str.replace('$', '', regex=False)  # Remove dollar signs
        .str.replace(',', '', regex=False)  # Remove commas
        .str.replace(' ', '')  # Remove spaces
        .str.replace('%', '')  # Remove spaces
        .str.strip()  # Trim whitespace
        .pipe(pd.to_numeric, errors='coerce')  # Convert to numeric
    )


exposure = exposure_raw.apply(clean_column)
exposure = exposure.groupby('ZIP Code').first().reset_index()

policy = policy_raw.apply(clean_column)
policy = policy[policy['ZIP Code']>1].set_index('ZIP Code').reset_index()


exposure_growth = exposure[['ZIP Code','YoY Growth 2024', 'YoY Growth 2023', 'YoY Growth 2022', 'YoY Growth 2021']]
exposure_total = exposure[['ZIP Code','Total Exposure 2024', 'Total Exposure 2023', 'Total Exposure 2022', 'Total Exposure 2021', 'Total Exposure 2020' ]]

policy_growth = policy[['ZIP Code','YoY Growth 2024', 'YoY Growth 2023', 'YoY Growth 2022', 'YoY Growth 2021']]
policy_total = policy[['ZIP Code','Total Policies 2024', 'Total Policies 2023', 'Total Policies 2022', 'Total Policies 2021', 'Total Policies 2020' ]]

exposure_growth

# np.array([policy['ZIP Code']].head(20))
# policy['ZIP Code'] = policy['ZIP Code'].astype(int)

# growth = policy.merge(exposure, on='ZIP Code')
# growth

,ZIP Code,YoY Growth 2024,YoY Growth 2023,YoY Growth 2022,YoY Growth 2021
0,90001.0,13.5,7.9,6.2,5.2
1,90002.0,18.7,6.1,6.8,1.9
2,90003.0,26.6,10.6,6.9,1.1
3,90004.0,32.1,25.1,1.2,1.7
4,90005.0,53.1,9.9,-3.0,16.9
...,...,...,...,...,...
1660,96146.0,17.9,23.7,21.9,90.4
1661,96148.0,24.9,58.5,50.6,110.7
1662,96150.0,27.3,42.2,27.8,57.3
1663,96161.0,24.7,40.7,55.7,61.0
